In [1]:
import gym 
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete
import numpy as np
import random
import os
import pandas as pd
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy

# Functie & Environment

In [2]:
def MakeShuffledContainerList(GridRows=3, GridColumns=3, BootAantal=3):
    #The same number of containers on all boats
    number_sample = list(range(1,BootAantal+1))*GridColumns
    #Shuffle the list
    return random.sample(number_sample, len(number_sample))

def MakeRandomContainerList(GridRows=3, GridColumns=3, BootAantal=3):
    Lijst = []
    
    #Fill list with random Containers
    while len(Lijst) != GridRows*GridColumns:
        x = random.randint(1, BootAantal)
        Lijst.append(x)
    
    #Always at least 1 container per boat
    for i in range(1, BootAantal):
        if i not in Lijst:
            return MakeRandomContainerList(GridRows, GridColumns, BootAantal)
    return Lijst

def checkForBoxedInContainer(rowList):
    numberToCheckFor = np.nan
    numberToVerify = np.nan
    reward = 0
    for item in rowList:
        #Skip empty position on grid
        if (item != 0):
            if (np.isnan(numberToCheckFor)):
                numberToCheckFor = item
            if (not np.isnan(numberToVerify)):
                if (item != numberToVerify):
                    #The previous item is boxed in since that item is not equal to where we started and the next item is also different
                    return -5
            #Set number to verify if boxed in
            if ((item != numberToCheckFor) & (np.isnan(numberToVerify))):
                numberToVerify = item

    return reward

In [3]:
class Containers(Env):
    def __init__(self, GridRows, GridColumns, numberOfShips):
        self.GridRows = GridRows
        self.GridColumns = GridColumns
        self.numberOfShips = numberOfShips
        self.state = pd.DataFrame(index=range(GridRows),columns=range(GridColumns)).fillna(0)
        print(type(self.state))
        self.observation_space = Box(low=int(0),high=int(numberOfShips) ,shape=(GridRows,GridColumns), dtype = int)
        self.action_space = MultiDiscrete([numberOfShips,numberOfShips])
        self.list_containers = MakeShuffledContainerList(GridRows, GridColumns, numberOfShips)
    
    def step(self, action):
        #Get a container from the container list
        container = self.list_containers.pop()
        #Get a preview on the position
        preview_cont = self.state.at[action[0], action[1]]
        
        #Reward if the position is empty
        if preview_cont == 0:
            reward = 1
        else:
            reward = -5
        
        #Reward if placed next to the same container
#-------LEFT
        if action[1] == 0:
            #Container Reight the same?
            if self.state.at[action[0], action[1]+1] == container: 
                reward += 1

#-------RIGHT
        elif action[1] ==  len(self.state.head())-1:
            #Container Left the same?
            if self.state.at[action[0], action[1]-1] == container: 
                reward += 1

#-------MIDDLE  
        else:
            #Container Left the same?
            if self.state.at[action[0], action[1]-1] == container:
                reward += 1
            #Container Right the same?
            if self.state.at[action[0], action[1]+1] == container:
                reward += 1
            
            #Container between 2 Containers? 
            #[0,0,X,0,0] Good
            #[1,0,X,0,1] Wrong, Cause it can only place on the sides (Even if it's a 1)
            if not (all(v == 0 for v in self.state.to_numpy()[action[0]][:action[1]]) \
                    or all(v == 0 for v in self.state.to_numpy()[action[0]][action[1]+1:])):
                reward -= 5

        #Placing the container
        self.state.at[action[0],action[1]] = int(container)

        
#-------EXTRA +=; Row fully filled, extra reward
        #First check if it is empty, otherwise the model will constantly place boxes on full rows
        if preview_cont == 0:
            if self.state.eq(self.state.iloc[:,1], axis=0).all(1)[action[0]] == True:
                reward += 5
        
#-------EXTRA +=; The more variation in boat numbers in a row, the worse the reward
        df = self.state.iloc[action[0]].to_numpy()
        if len(set(df[df!=0])) > 2:
            reward -= 2
        
        #When a contianer is being boxed in
        reward += checkForBoxedInContainer(df)
        
        #If the ContainerList is empty, it stops
        if len(self.list_containers) == 0:
            done = True
        else:
            done = False
        
        info = {}
        return self.state.to_numpy(), reward, done, info
    
    #Display the environment
    def render(self, mode='human', close = False):
        display(self.state)
    
    #Set the state (grid) en the containerlist to the default states
    def reset(self):
        self.state = pd.DataFrame(index=range(self.GridRows),columns=range(self.GridColumns)).fillna(0)
        self.list_containers = MakeShuffledContainerList(self.GridRows, self.GridColumns, self.numberOfShips)
        return self.state.to_numpy()

# Test Model

In [4]:
GridRows=4
GridColumns=4
numberOfShips=4
Env = Containers(GridRows, GridColumns, numberOfShips)

# Initialize the state of the environment
state = Env.reset()

# Number of episodes to run the simulation for
episodes = 20

# Loop through the episodes
for episode in range(1, episodes+1):
    # Starting values for the episode
    done = False
    score = 0 
    
    # Loop through the actions within an episode
    while not done:
        # Render the environment
        Env.render()
        # Sample an action from the action space
        action = Env.action_space.sample()
        # Print the row and column of the action
        print("Row, Column: ", action[0], action[1])
        # Take the action and get the new state, reward, done status, and additional information
        n_state, reward, done, info = Env.step(action)
        # Add the reward to the score
        score+=reward
    # Print the episode number and score at the end of each episode
    print('Episode:{} Score:{}'.format(episode, score))
    # Reset the environment for the next episode
    state = Env.reset()

# Close the environment
Env.close()

<class 'pandas.core.frame.DataFrame'>


,0,1,2,3
0,0,0,0,0
1,0,0,0,0
2,0,0,0,0
3,0,0,0,0


Row, Column:  1 0


,0,1,2,3
0,0,0,0,0
1,1,0,0,0
2,0,0,0,0
3,0,0,0,0


Row, Column:  0 3


...

Row, Column:  1 2


,0,1,2,3
0,0,0,0,4
1,1,0,2,2
2,0,0,1,1
3,4,0,4,3


Row, Column:  0 2


,0,1,2,3
0,0,0,3,4
1,1,0,2,2
2,0,0,1,1
3,4,0,4,3


Row, Column:  0 1
Episode:20 Score:-28


# Train Model

In [5]:
model = PPO("MlpPolicy", Env, verbose=1)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/opt/jupyterhub/anaconda/lib/python3.9/site-packages/torch/cuda/__init__.py:83: UserWarning: CUDA initialization: CUDA driver initialization failed, you might not have a CUDA gpu. (Triggered internally at  /opt/conda/conda-bld/pytorch_1659484809662/work/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


In [6]:
model.learn(total_timesteps= 2_000_000)

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 16       |
|    ep_rew_mean     | -46.9    |
| time/              |          |
|    fps             | 984      |
|    iterations      | 1        |
|    time_elapsed    | 2        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 16          |
|    ep_rew_mean          | -44.3       |
| time/                   |             |
|    fps                  | 787         |
|    iterations           | 2           |
|    time_elapsed         | 5           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.009306009 |
|    clip_fraction        | 0.0654      |
|    clip_range           | 0.2         |
|    entropy_loss         | -2.77       |
|    explained_variance   | 0.00361     |
|    learning_rate        | 0.

In [7]:
from stable_baselines3.common.monitor import Monitor
Env.reset()
monitor = Monitor(Env)
evaluate_policy(model, monitor, n_eval_episodes=5, return_episode_rewards= True, render=True)

,0,1,2,3
0,0,0,0,0
1,0,0,0,0
2,0,4,0,0
3,0,0,0,0


,0,1,2,3
0,0,0,0,0
1,0,0,0,0
2,0,4,0,0
3,0,2,0,0


,0,1,2,3
0,0,0,0,0
1,0,2,0,0
2,0,4,0,0
3,0,2,0,0


,0,1,2,3
0,0,0,0,0
1,0,2,0,0
2,0,4,0,0
3,0,2,2,0


,0,1,2,3
0,0,0,4,0
1,0,2,0,0
2,0,4,0,0
3,0,2,2,0


...

,0,1,2,3
0,0,0,3,1
1,0,1,0,2
2,0,4,4,2
3,0,4,3,4


,0,1,2,3
0,0,0,3,2
1,0,1,0,2
2,0,4,4,2
3,0,4,3,4


,0,1,2,3
0,0,0,3,3
1,0,1,0,2
2,0,4,4,2
3,0,4,3,4


,0,1,2,3
0,0,0,0,0
1,0,0,0,0
2,0,0,0,0
3,0,0,0,0


([-23, -19, -10, -17, -22], [16, 16, 16, 16, 16])